## import libraries

In [1]:
import numpy as np

## Problem setup

In [2]:
classes = [f'C{i+1}' for i in range(12)]
professors = {
    'C1': 'P1', 'C2': 'P1',
    'C3': 'P2', 'C4': 'P2',
    'C5': 'P3', 'C6': 'P3',
    'C7': 'P4', 'C8': 'P4',
    'C9': 'P5', 'C10': 'P5',
    'C11': 'P6', 'C12': 'P6'
}
timeslots = [f'T{i+1}' for i in range(6)]
n_classes = len(classes)
n_times = len(timeslots)
max_classes_per_time = 3


## Constraints

In [3]:
# Professors availability (each can only teach in some time slots)
availability = {
    'P1': [0, 1, 2],
    'P2': [1, 2, 3],
    'P3': [2, 3, 4],
    'P4': [0, 4, 5],
    'P5': [0, 2, 5],
    'P6': [1, 3, 5]
}

# Class pairs that cannot overlap (students overlap)
cannot_overlap = [('C1', 'C3'), ('C4', 'C6'), ('C2', 'C8'), ('C5', 'C10'), ('C7', 'C11')]


## Initialize pheromones and heuristic

In [4]:
#TODO: initialze these parameters

# Initialize pheromone with small values
pheromone = np.full((n_classes, n_times), 0.1)

# Initialize heuristic as 1 (neutral), could be improved by analyzing fewer conflicts
heuristic = np.ones((n_classes, n_times))

## Ant builds a solution

In [5]:
#TODO: complete construct_solution function
def construct_solution(pheromone, heuristic, alpha=1, beta=2):
    schedule = []
    for i in range(n_classes):  # For each class
        probs = []
        for t in range(n_times):
            tau = pheromone[i][t] ** alpha
            eta = heuristic[i][t] ** beta
            probs.append(tau * eta)
        probs = np.array(probs)
        probs /= probs.sum()
        time = np.random.choice(range(n_times), p=probs)
        schedule.append(time)
    return schedule

## Evaluate constraints

In [6]:
def evaluate_conflicts(schedule):
    conflicts = 0
    time_table = {t: [] for t in range(n_times)}
    prof_time = {t: [] for t in range(n_times)}

    for i, time in enumerate(schedule):
        cls = classes[i]
        prof = professors[cls]
        time_table[time].append(cls)
        prof_time[time].append(prof)

    # 1. Max 3 classes per time
    for t in range(n_times):
        if len(time_table[t]) > max_classes_per_time:
            conflicts += len(time_table[t]) - max_classes_per_time

    # 2. Professor assigned to more than 1 class in the same timeslot
    for t in range(n_times):
        profs = prof_time[t]
        for p in set(profs):
            count = profs.count(p)
            if count > 1:
                conflicts += count - 1

    # 3. Professor availability
    for i, time in enumerate(schedule):
        cls = classes[i]
        prof = professors[cls]
        if time not in availability[prof]:
            conflicts += 1

    # 4. Student overlap
    assigned_times = {cls: schedule[i] for i, cls in enumerate(classes)}
    for c1, c2 in cannot_overlap:
        if assigned_times[c1] == assigned_times[c2]:
            conflicts += 1

    return conflicts

## Pheromone update

In [7]:
def update_pheromone(pheromone, solutions, scores, rho=0.1, Q=100):
    # Evaporation
    pheromone *= (1 - rho)

    for schedule, score in zip(solutions, scores):
        # add small epsilon to avoid divide by zero
        deposit = Q / (score + 1e-6)
        for i in range(n_classes):
            pheromone[i][schedule[i]] += deposit

## ACO main loop

In [8]:
n_ants = 10
n_iter = 30
best_sol = None
best_score = float('inf')
history = []

for it in range(n_iter):
    solutions = [construct_solution(pheromone, heuristic)
                 for _ in range(n_ants)]
    scores = [evaluate_conflicts(s) for s in solutions]

    update_pheromone(pheromone, solutions, scores)

    min_score = min(scores)
    min_sol = solutions[np.argmin(scores)]

    if min_score < best_score:
        best_score = min_score
        best_sol = min_sol

    history.append(best_score)
    print(f"Iteration {it+1}, Best score: {best_score}")

Iteration 1, Best score: 5
Iteration 2, Best score: 5
Iteration 3, Best score: 5
Iteration 4, Best score: 5
Iteration 5, Best score: 3
Iteration 6, Best score: 3
Iteration 7, Best score: 2
Iteration 8, Best score: 2
Iteration 9, Best score: 2
Iteration 10, Best score: 2
Iteration 11, Best score: 2
Iteration 12, Best score: 2
Iteration 13, Best score: 2
Iteration 14, Best score: 2
Iteration 15, Best score: 2
Iteration 16, Best score: 2
Iteration 17, Best score: 2
Iteration 18, Best score: 2
Iteration 19, Best score: 2
Iteration 20, Best score: 0
Iteration 21, Best score: 0
Iteration 22, Best score: 0
Iteration 23, Best score: 0
Iteration 24, Best score: 0
Iteration 25, Best score: 0
Iteration 26, Best score: 0
Iteration 27, Best score: 0
Iteration 28, Best score: 0
Iteration 29, Best score: 0
Iteration 30, Best score: 0


## show result


In [9]:
print("\nBest schedule with minimum conflicts:")
for i, time in enumerate(best_sol):
    print(f"Class {classes[i]} → Time {timeslots[time]}")
print("Total conflicts:", best_score)


Best schedule with minimum conflicts:
Class C1 → Time T1
Class C2 → Time T2
Class C3 → Time T4
Class C4 → Time T2
Class C5 → Time T3
Class C6 → Time T4
Class C7 → Time T5
Class C8 → Time T6
Class C9 → Time T3
Class C10 → Time T1
Class C11 → Time T4
Class C12 → Time T2
Total conflicts: 0
